# $SPHINCS^+$ Python : Example #

### Importing $SPHINCS^+$ Class ###

In [1]:
from package.sphincsc import SphincsC

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

By default, $SPHINCS^+$ parameters are set like:
   - Security Parameter: $n=16$
   - Winternitz Parameter: $w=16$ (Should be set as 4, 16 or 256)
   - Hypertree Height: $h=64$
   - Hypertree Layers: $d=8$ ($d|h$)
   - $FORS$ Trees Number: $k=10$
   - $FORS$ Trees Height: $a=15$

Changes must be done before keys generations, signing and verification

In [7]:
sphincs = SphincsC()
sphincs.set_n(16)
sphincs.set_h(63)
sphincs.set_d(21)
sphincs.set_k(19)
sphincs.set_w(16)
sphincs.set_a(15)
sphincs.set_cf(4)


### Generating a Key Pair ### 

In [8]:
sk, pk = sphincs.generate_key_pair()
print("Secret Key: ", sk)
print()
print("Public Key: ", pk)

Secret Key:  b'\x02\xafx\xf9rD\xe6\x97\xf4=\x1d\x87\xa2\x0fTs\x1f\x8d\x17\xa9\xc8\\S\x17\x9c<\xbb\x8f\x97L\xdb\x90I:|\xae1\x94G\x9f\x81\xcd\x8e{Y{M\r_\r\x0f\x1au\xaa\xda\xc8Gj_\xad,\xa8b\x86'

Public Key:  b'I:|\xae1\x94G\x9f\x81\xcd\x8e{Y{M\r_\r\x0f\x1au\xaa\xda\xc8Gj_\xad,\xa8b\x86'


In [11]:
len(sk)

64

In [12]:
type(sk)

bytes

### Signing a Message ###

The message must be in a $bytes()$ format

In [9]:
# m = b"No one knows the reason for all this, but it is probably quantum. - Pyramids, Terry Pratchett (1989)"
m = b'Ripples of paradox spread out across the sea of causality.'
print("Message to be signed: ", m)

Message to be signed:  b'Ripples of paradox spread out across the sea of causality.'


In [10]:
signature = sphincs.sign(m, sk)

In [ ]:
signature

b'3\xdc\x7fS*\xea\xf58\x1f\x0f9\x8d\xddyM\x9a\\\x8e{\x11\x90\xf0D\x0b:K\x18w\xe7^\x85%\xdc\xea\xb9\xb8se\x03D\xe3\xe8\xe5S%E-\x0e\x04Q^{\xc8\x91\x8b\x1f\xf7\xd7\xc7\xefw>;D|\xb2\xf9\x8eLIT\xee\xe6\x11\x97s\xf5k@_\xbe\t\xe5\xb5:Ah\x0f\x06\x87\x1dx\x1e\x8f\xf6\x11*K\x87\xc72LJ\xdb\xdf\x9e\xcbb\x16t8\x9f\x0b\x98\x1f\x8c\x90>\xac7<\\G\xbeH\x94\xdf\xe4\x1e\xde0@\x1d\x18\xe6\xbf*\x9c\xa8EFT\x8b\\(\x1eK(p\xa0\x12Z\xfe\xff\xe8e\x1f\x92<\x8a\xde\x1f\x0f\xad\t\xcb\x8d\xe9\xb30\xbc\xf3\x0c\xb9\xd9\x163\x16\xf5\x8f\xcc\xe1\x15\r\x8f\x1e\x8e\x14\xf61\xb8\x12\x19\x82\x0b\xc4\xc1;{\xd1\xe5N\x82\xa1\xea\x9e\x86l`m--\x0c\x0f\xa2\x94\xfe\xc1g"\xa5\xb7\xachNR\xfa\x98 Hg|\x84\t\x04\xb4\\\xc5[L\x9a\x05t\xb8\x82dL\xfe/b\xe1{h\x83B?\xb8\xb19\xc3\x18Pbl0\x8f\x0b\x90i\xfbC\xb6\xa8\x8bf\xc6\x9b,0\xe70H\xcc\xa8M-i+\xdakD\n\xdbF\xe8\xbd\xa8\xbf\xe6\x15\xae\xd0\x04O\x9a\x1e\xb6\x99\x8a\x83+n\x7f]{\xf4\xf6\x8a\xcc\xed_b#\xafS\xa1Q\xdb3F\xf7\x99\x87u\xeei\xff\x9c\xfa\xbd\x1b\x8e\xd4\x0f=\xac\xa92>li\xeb\x91\xd2\x1c\

In [7]:
print("Signature bytes-size: ", len(signature))

Signature bytes-size:  7401


### Verifying a Signature associated with a Message ###

In [8]:
print("Is signature correct ? ", sphincs.verify(m, signature, pk))

Is signature correct ?  True


In [ ]:
sphincs.set_z(2)

In [ ]:
signature2 = sphincs.sign(m, sk)
print("Signature bytes-size: ", len(signature2))

In [ ]:
print("Is signature correct ? ", sphincs.verify(m, signature2, pk))

### Trying to find secret key with a Brute Force Attack on Secret Key

In [9]:
sk_crack = bytes()
end_val = 2 ** (sphincs._n * 8)

for i in range(0, 2 ** (sphincs._n * 8)):
    print(f"Value of i is {i} of {end_val}", end="\r")
    sk_crack = i.to_bytes(sphincs._n, 'big')  # Secret Key

    # Random Secret PRF, important to prevent forged messages from actual messages
    sk_crack += bytes(sphincs._n)
    # But Because we are brute forcing Secret Key, messages are forged before
    # We don't need to create a really random one (0 is fine)
    sk_crack += pk  # Public Key

    sig_crack = sphincs.sign(m, sk_crack)  # Creating a signature

    # Check if signature could be trust with the Public Key
    if sphincs.verify(sig_crack, m, pk):
        print("Secret Key Found: ", sk_crack,
              "\nWith main Private Seed: ", sk_crack[:sphincs._n])
        print("Cycles: ", i)
        break

print("\nDid we found the actual Private Seed? ",
      sk[:sphincs._n] == sk_crack[:sphincs._n])

if sk[:sphincs._n] != sk_crack[:sphincs._n]:
    print("We found a collision with the main seed!")


KeyboardInterrupt: 

### Trying to forge message with found Key ###

In [ ]:
m2 = b'The pen is mightier than the sword ... if the sword is very short, and the pen is very sharp.'

signature2 = sphincs.sign(m2, sk_crack)

print(len(signature2))


In [ ]:
print("Is signature Correct ? ", sphincs.verify(signature2, m, pk))


Our signature is wrong because we tried finding the secret key using only $m$ signature, with $m_2$, this kind of collision doesn't work !
We need to find the real secret key in order to forge our own message, and so test every possibilities.

### Graft Fault Attack on $SPHINCS^+$ Framework

We will try to attack $SPHINCS^+$ framework by grafting a fake tree on the main hypertree by exploiting an error generated by the signing alogirthm as it can be explained in this document : https://eprint.iacr.org/2018/102.pdf

### Instantiate a $SPHINCS^+$ Object and Setting parameters ###

We will be using :
   - Security Parameter: $n=2$
   - Winternitz Parameter: $w=16$
   - Hypertree Height: $h=4$
   - Hypertree Layers: $d=2$
   - $FORS$ Trees Number: $k=4$
   - $FORS$ Trees Height: $a=2$

In [ ]:
sphincs = Sphincs()

# sphincs.set_n(2)
# sphincs.set_h(4)
# sphincs.set_d(2)
# sphincs.set_k(4)
# sphincs.set_a(2)

We will suppose that a message has been signed twice while a flaw in the algorithm made the second signature with a faulty authentification path with private key _b'mo\x9c\x1bS\xd4\x05\x16'_

In [ ]:
# Public key
pk = b'S\xd4\x05\x16'


In order to simplify this attack, me used the same $R$ variable so we could be sure our two signatures will use the same path of authentification

In [ ]:
m = b'Not our fault if it breaks...'
sig1 = b'\x00\x00QUS\xff\x90\x94uvD\xa1\xd8\xce\xa0\xa8\xd2\x8b`9\x84\x10\x07-i\x94\xfe3\xdb\x97\x00\xc8E\xd3\xdd\x14\x15\x81n\x13\x1f\x8d\xc3?\xee\xfe\xb0\xa0puV\xb6OEH;\xfe\xe1'
sig2 = b'\x00\x00QUS\xff\x90\x94uvD\xa1\xd8\xce\xa0\xa8\xd2\x8b`9\x84\x10\x07-i\x94\xfe3\xdb\x97\x00\xc8E\xd3\xdd\x14\x15\x81n\x13\x00\x00r\x05\xca\xb3P\xfd\x11\xb1\xbbN\r\xd8H;\xfe\xe1'


In [ ]:
# Splitting SIGs in order to find wots message and signature
h_prime = sphincs._h_prime
len_0 = sphincs._len_0
len_1 = sphincs._len_1
n = sphincs._n
d = sphincs._d
a = sphincs._a
k = sphincs._k
h = sphincs._h


def sigs_from_main_sig(sig):
    sig_tab = []

    sig_tab += [sig[:n]]  # R

    sig_tab += [[]]  # SIG FORS
    for i in range(n,
                   n + k * (a + 1) * n,
                   n):
        sig_tab[1].append(sig[i:(i + n)])

    sig_tab += [[]]  # SIG Hypertree
    for i in range(n + k * (a + 1) * n,
                   n + k * (a + 1) * n + (h + d * len_1) * n,
                   n):
        sig_tab[2].append(sig[i:(i + n)])
    return sig_tab


def sig_wots_from_sig_xmss(sig):
    return sig[0:len_1]


def auth_from_sig_xmss(sig):
    return sig[len_1:]


def sigs_xmss_from_sig_ht(sig):
    sigs = []
    for i in range(0, d):
        sigs.append(sig[i * (h_prime + len_1):(i + 1) * (h_prime + len_1)])

    return sigs


splitted_sig1 = sigs_from_main_sig(sig1)
splitted_sig2 = sigs_from_main_sig(sig2)
xmss_sig1 = sigs_xmss_from_sig_ht(splitted_sig1[2])
xmss_sig2 = sigs_xmss_from_sig_ht(splitted_sig2[2])

# WOTS signature from the XMSS tree d-1 is signing the previous auth from tree d-2
print("Auth XMSS d-2 sig1: ", auth_from_sig_xmss(xmss_sig1[0]))
print("WOTS SIG d-1  sig1: ", sig_wots_from_sig_xmss(xmss_sig1[1]))
print("Auth XMSS d-2 sig2: ", auth_from_sig_xmss(xmss_sig2[0]))
print("WOTS SIG d-1  sig2: ", sig_wots_from_sig_xmss(xmss_sig2[1]))
print()
print("Auth XMSS d-1 sig1: ", auth_from_sig_xmss(xmss_sig1[1]))
print("Auth XMSS d-1 sig2: ", auth_from_sig_xmss(xmss_sig2[1]))


As we can see, our second signature has a faulted authentification path, different from the first signature. Because of this, we obtain an other signature from the same $WOTS^+$ leaf. But even if the signature is different, because it has been successfully signed, it is counted as a true signature:

In [ ]:
print("Is sig1 correct? ", sphincs.verify(sig1, m, pk))
print("Is sig2 correct? ", sphincs.verify(sig2, m, pk))


We need to find a random _seed_ for which our forged message go through the same $WOTS^+$ leaf. To do this, we need to find a $R$ that the verifying algorithm will use to go through the attacked leaf.

In [ ]:
m_fake = b'The earth is flat!'

# Here for this hypertree and this fake message, we know that if R = b'a\x06', we will go through the same leaf on d-1 tree
r = b'a\x06'


This value has been easily found because we can still sign our own message with the secret key (we could then obtain the message signature: b'a\x06\xe1\xe6\xea\xfb\xbd)\xd0+\x9b\xd7+\x94\xd8Vk\xcct\x9fzl\xf1\xe4\xb1%\xcfD\xf4\xa5\xa8\x1e\x14\xa7#\x1a\xe2\xef\xd9\x08\x1f\x8d\xc3?\xee\xfe\xb0\xa0puV\xb6OEH;\xfe\xe1'). But knowing which path is used to verify the signature is publicly known: it depends from $pk$, $m$ and $R$ which are publicly known when we digest the message in the first verification step.

From these informations, we need to compute the $m_{fake}$ signature with different sub-hypertrees (size $h - h'$ with $d-1$ layers) until we can found a tree for which his computed root can be signed with the known $WOTS^+$ leaf. The more faulted signature we have, the more possible roots can be forged. 

From this part, we have to brute force sub-hypertrees possibilities until we found one that can be used with this $WOTS^+$ key. We can then sign our message with our $R$, our tree and we then add the final official authentification path to the signature to make it computable by others.